## Modelos de Machine Learning
Instalamos las dependencias necesarias: numpy, pandas, scikit-learn

In [2]:
%pip install numpy
%pip install pandas
%pip install scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Importamos las librerias que vamos a usar

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor


Numpy version: 2.0.0
Pandas version: 2.2.2


In [17]:
file_path = 'data.csv'
data = pd.read_csv(file_path)

y = data[["SA", "SBR", "O (%)"]]

print(y)

      SA   SBR  O (%)
0   2.33  1.10  42.90
1   2.33  1.09  43.41
2   2.33  1.11  42.60
3   2.33  1.14  35.26
4   2.33  1.03  34.73
5   2.33  1.16  41.88
6   2.33  1.22  39.46
7   2.33  1.15  42.30
8   2.33  1.23  39.00
9   2.33  1.22  38.99
10  2.33  2.11  42.90
11  2.33  2.10  43.41
12  2.33  2.14  42.60
13  2.33  2.18  35.26
14  2.33  1.98  34.73
15  2.33  2.22  41.88
16  2.33  2.34  39.46
17  2.33  2.21  42.30
18  2.33  2.37  39.00
19  2.33  2.34  38.99
20  2.33  1.22  38.99
21  2.33  1.22  38.99
22  0.11  0.06  38.99
23  0.33  0.17  38.99
24  1.00  0.52  38.99
25  2.33  2.34  38.99
26  2.33  2.34  38.99
27  0.11  0.11  38.99
28  0.33  0.33  38.99
29  1.00  1.00  38.99


In [33]:
X = data[['SA', 'SBR', 'T (K)', 'SR', 'C (%)', 'N (%)', 'H (%)', 'O (%)']]
y = data['volH2 (%)']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cross_val_score(LinearRegression(), X_train, y_train, cv=5)

best_model = LinearRegression()

best_model.fit(X_train, y_train)
test_score = best_model.score(X_test, y_test)

print("Test score: ", test_score)

# Predict the test data
y_pred = best_model.predict(X_test)
print(y_test)
print(y_pred)

Test score:  0.8343017494622117
27    27.8254
15    31.1787
23    29.7194
17    32.2669
8     36.0748
9     35.5075
Name: volH2 (%), dtype: float64
[29.35042049 32.77084566 30.65565443 32.69602229 34.42213567 35.47378417]


In [47]:
X = data[['SA', 'SBR', 'T (K)', 'SR', 'C (%)', 'N (%)', 'H (%)', 'O (%)']]
y = data['volH2 (%)']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cross_val_score(LinearRegression(), X_train, y_train, cv=5)

best_model = LinearRegression()

best_model.fit(X_train, y_train)
test_score = best_model.score(X_test, y_test)
scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')


print("Test score: ", test_score)
print(f"Cross validation scores: {-np.mean(scores):.3f}",)

# Predict the test data
y_pred = best_model.predict(X_test)
print(y_test)
print(y_pred)

Test score:  0.8343017494622117
Cross validation scores: 5.835
27    27.8254
15    31.1787
23    29.7194
17    32.2669
8     36.0748
9     35.5075
Name: volH2 (%), dtype: float64
[29.35042049 32.77084566 30.65565443 32.69602229 34.42213567 35.47378417]
